# Two-port Sparameter Graph

## Models that fit into this 
1. S2P on disk 
2. SNP / S2PV1 Classes in Touchstone Models
3. Jim Booth Format on Disk -> JBSparameter class
4. TwoPortNRRawModel and TwoPortRawModel
5. S2P on disk in different formats (AsciiDataTable)
6. XMLDataTables
7. XsltResultString an XslResultFile
8. CITI
9. VNATools

## Issues
1. When doing analysis the data must present a uniform format, i.e. units same, values are complex matrix or list
2. Header or Metadata is removed for processing
3. What is the result of a measurement or the result of a model or simulation

## Common Analysis
1. form a correction given a series of measured standards (requires more than one measurement, definition and some extra metadata)
2. Apply a correction to a uncorrected DUT measurement
3. Extract device parameters given the corrected DUT measurement, this assumes DUT fits a model
4. Track or form Uncertainty estimates in 1-3



In [1]:
from pyMeasure import *
from pyMeasure.Code.DataHandlers.GraphModels import *

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [ ]:
def S2pFile_to_S2p(s2p_file_name):
    """Returns a S2p class given a S2p file name"""
    return S2PV1(s2p_file_name)

def S2p_to_S2pFile(s2p,s2p_file_name="test.s2p"):
    """Saves the s2p class as file_name"""
    s2p.save(s2p_file_name)
    return s2p_file_name

In [ ]:
graph_options={"graph_name":"Two Port Graph",
                  "node_names":["S2pFile",'S2pV1'],
                  "node_descriptions":["S2P file on disk","S2p class in Touchstone.Models"],
                  "current_node":'S2pFile'
                  "state":[1,0],
                  "data":"test.s2p"
                  "edge_2_to_1":S2p_to_S2pFile
                  "edge_1_to_2":S2pFile_to_S2p}